In [12]:
# Standard Library Packages
import subprocess
from io import BytesIO

# Other Packages
import folium
import holoviews as hv
import hvplot.pandas
import pandas as pd
import requests
import warnings


# Ignore warnings
warnings.filterwarnings("ignore")

# Streamflow in the Burlington Area
### Site Description

<figure style="display: inline-block; border: 1px dotted gray; margin: 20px;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/3/39/Connecticut_River_Map.png" style="float:left; height: 500px; vertical-align: top;" alt='missing'/>
    <figcaption style="text-align: center; height: 400px; vertical-align: top" >Connecticut River Watershed
        <br><a href="https://commons.wikimedia.org/wiki/File:Connecticut_River_Map.png">USGov</a>, Public domain, via Wikimedia Commons
        </figcaption>
</figure>

<figure style="display: inline-block; margin: 20px;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/e/ea/Connecticut_River_-_Windsor_Locks_%2832247230703%29.jpg" style="float:left; height: 500px; vertical-align: top;" alt='missing'/>
    <figcaption style="text-align: center; height: 400px; vertical-align: top" >Connecticut River in Windsor Locks, CT
        <br>Source: <a href="https://commons.wikimedia.org/wiki/File%253AConnecticut_River_-_Windsor_Locks_%252832247230703%2529.jpg">Photo</a>
 by formulanone <a href="https://creativecommons.org/licenses/by-sa/2.0/deed.en"> CC BY 2.0</a> 
        </figcaption>
</figure>

#### Lewis Creek in North Ferrisburg, VT

In [13]:
# Define Latitute and Longitude
sg_lat = 44.249167
sg_lon = -73.228889


# Initialize map and tweak settings
frame = folium.Figure(height=500, width=775)
m = folium.Map(width="100%", height="100%", zoom_start=7,
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False).add_to(frame)

# Put a marker at the stream gauge location
folium.Marker(
    [sg_lat, sg_lon], popup="Lewis Creek at North Ferrisburg, VT"
    ).add_to(m)

# Display the map
m

In [54]:
# Get Omaha Data (using ChatGPT suggested code)
lewis_Data_url = ("https://waterdata.usgs.gov/nwis/dv"
               "?cb_00060=on"
               "&format=rdb"
               "&site_no=04282780"
               "&legacy="
               "&referred_module=sw"
               "&period=&"
               "begin_date=1990-03-01"
               "&end_date=2023-09-21")

# Send an HTTP GET request to the URL
lewis_response = requests.get(lewis_Data_url)

# Check if the request was successful (status code 200)
if lewis_response.status_code == 200:
    # Print the content of the response (HTML, JSON, etc.)
    print(lewis_response.text)
else:
    print(f"Request failed with status code {lewis_response.status_code}")

lewis_response

# ---------------------------------- WARNING ----------------------------------------
# Some of the data that you have obtained from this U.S. Geological Survey database
# may not have received Director's approval. Any such data values are qualified
# as provisional and are subject to revision. Provisional data are released on the
# condition that neither the USGS nor the United States Government may be held liable
# for any damages resulting from its use.
#
# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_waterdata_support@usgs.gov
# retrieved: 2023-09-22 10:20:18 EDT       (sdww01)
#
# Data for the following 1 site(s) are contained in this file
#    USGS 04282780 LEWIS CREEK AT NORTH FERRISBURG, VT
# ----------------------------------------------------

<Response [200]>

In [55]:
lewis_daily_df = pd.read_csv(
    BytesIO(lewis_response.content),
    comment='#',
    delimiter='\t',
    skiprows=31,
    names=["Agency", "Site_Number", "Date", "Streamflow_cfs", "cd"],
    index_col='Date',
    parse_dates=True,
    na_values=["Ice", "Bkw"]
)

lewis_daily_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12258 entries, 1990-03-01 to 2023-09-21
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Agency          12258 non-null  object 
 1   Site_Number     12258 non-null  int64  
 2   Streamflow_cfs  12214 non-null  float64
 3   cd              12257 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 478.8+ KB


### Since 1928, the largest two floods in the Connecticut River in Thompsonville, CT both occurred in the 1930s.
USGS has measured streamflow at this site since 1928. The chart below shows the maximum value for average daily streamflow (cubic feet per second) by year, dating back to 1928.

In 1936 and 1938, the two largest streamflow values occurred, both of which exceeded (200,000 cubic feet per second).

In [56]:
# Resampling from daily to annual
lewis_annual_df = lewis_daily_df[['Streamflow_cfs']].resample('AS').max()

lewis_annual_plt = lewis_annual_df.hvplot(y='Streamflow_cfs', responsive= True, aspect=6/2).opts(
    title="Maximum Streamflow for the Connecticut River in Lewis Creek, VT (1928 to 2023)",
    xlabel="Year",
    ylabel="Streamflow (ft^3/s)",
    min_width=300,
    min_height=300,
)

lewis_annual_plt

:Curve   [Date]   (Streamflow_cfs)

### Maximum Streamflow since the Completion of Dams in 1971
In 1971, the US Army Corps of Engineers finished completion of 16 dams that were built in the wake of the 1936 and 1938 floods. Most of these 16 additional dams were in place by 1965.

The chart below shows maximum streamflow after the completion of these dams in 1971. 

In [61]:
lewis_daily_jul23_df = lewis_daily_df['2023-07-01':'2023-07-31']
lewis_daily_jul23_df.hvplot(y="Streamflow_cfs")

:Curve   [Date]   (Streamflow_cfs)

In [6]:
# Data from the period after dams were built.
ct_annual_df_recent = ct_annual_df["1971-01-01":"2023-01-01"].copy()

# Plot of Annual Streamflow
# Making size of hvplot responsive (https://github.com/holoviz/hvplot/issues/350)
ct_post1950_plt = ct_annual_df_recent.hvplot(y='Streamflow_cfs',responsive= True, aspect=6/2).opts(
    title="Maximum Streamflow for the Connecticut River in Thompsonville, CT (1971 to 2023)",
    xlabel="Year",
    ylabel="Streamflow (ft^3/s)",
    min_width=300,
    min_height=300
)

ct_post1950_plt

:Curve   [Date]   (Streamflow_cfs)

In [7]:
# Add exeedence probabilities
ct_annual_df_recent['exceed_prob'] = (
    ct_annual_df_recent
    .rank(ascending=False)
    / len(ct_annual_df_recent)
)

# Add Return Rate
ct_annual_df_recent['return_rate'] = (
    1 / ct_annual_df_recent['exceed_prob']
)

In [8]:
# Largest Floods
high_return_rates = ct_annual_df_recent[ct_annual_df_recent['return_rate'] > 10]
high_return_rates

ct_annual_df_recent['Rank'] = (
    ct_annual_df_recent['Streamflow_cfs']
    .rank(ascending=False))

ct_top6_df = ct_annual_df_recent[ct_annual_df_recent['Rank'] < 7]

In [9]:
# Prepare Dataframe for Printing
# Extract Year Portion of Date
ct_top6_df['Year'] = ct_top6_df.index.year

# Rename Columns for Printing
ct_top6_df = ct_top6_df.rename(columns={"Streamflow_cfs": "Streamflow (Cubic feet per second)", "exceed_prob": "Probability",
                     "return_rate": "Return Period (Years)"})

# Reset Index to Rank and Sort
ct_top6_df = ct_top6_df.set_index('Rank')
ct_top6_df = ct_top6_df.sort_values('Rank')

### The 2023 floods had the 6th largest streamflow of all floods since dam completion in 1971.

In [10]:
# Display Table.
ct_top6_df.style.format({ # Change formats for display (from: https://towardsdatascience.com/make-your-tables-look-glorious-2a5ddbfcc0e5)
    "Rank": "{:.0f}",
    "Streamflow (Cubic feet per second)": "{:,.0f}",
    "Probability": "{:,.2f}",
    "Return Period (Years)": "{:.0f}",
    "Rank": "{:,.0f}"
}).set_table_styles( # Add Highlighter for Row (from: https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.set_table_styles.html)
    [{'selector': 'tr:hover',
      'props': [('background-color', '#d0f0c0')]}]
).format_index('{:.0f}') # Reformat index (from: https://pandas.pydata.org/docs/dev/reference/api/pandas.io.formats.style.Styler.format_index.html)


,Streamflow (Cubic feet per second),Probability,Return Period (Years),Year
Rank,,,,
1,"176,000",0.02,53,1984
2,"145,000",0.04,26,1987
3,"122,000",0.06,18,2011
4,"112,000",0.08,13,1977
5,"111,000",0.09,11,1981
6,"109,000",0.11,9,2023



<figure style="display: inline; float: right; margin: 75px;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/8/88/1936Flood_HartfordCT01.jpg" style="float:right; height: 300px; vertical-align: top;" alt='missing'/>
    <figcaption style="text-align: right; vertical-align: top" >1936 Connecticut River Flood in Downtown Hartford, CT   
        <br>Source: <a href="https://commons.wikimedia.org/wiki/File:1936Flood_HartfordCT01.jpg">John F. Dreier</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a>, via Wikimedia Commons
        </figcaption>
</figure>


### Summary
__2023 Floods__: 
  * The flooding in 2023 that has adversely affected farmers in Connecticut is the largest flood in a dozen years and the 6th largest since 1971. However, it is not an extremely unusual event (return rate of 9). 

  * While not a major flood in terms of streamflow, it has caused flooding in low-lying farmland in northern/central Connecticut and had substantial impact due to its timing right before the start of harvesting. 

__Building of Dams and the Impact on Streamflow__: 
  * There were two major floods in 1936 and 1938. The US Army Corps of Engineers subsequently built 16 flood-control dams in the Connecticut River between 1940 and 1971. 

  * Streamflow has never surpassed 200,000 cubic feet per second at this location since that time.
  
  * There were two additional large floods in 1984 and 1987, which were the largest since the building of dams, though still smaller than the 1930s floods.

### References
* US Army Corps of Engineers (n.d.). May/June 1984 Flood. Retrieved September 16, 2023 from: https://reservoircontrol.usace.army.mil/nae_ords/cwmsweb/utility.g?p_path%253D1984.pdf
* Skahill, P. (2023, July 13). Connecticut River flooding destroys farms, just as harvest season begins. ‘What am I going to do?. Connecticut Public. https://www.ctpublic.org/news/2023-07-13/connecticut-river-flooding-destroys-farms-just-as-harvest-season-begins-what-am-i-going-to-do
* Savitt, M. (July 14, 2023). Connecticut River seeing water quality issues amid flooding. Connecticut Public. https://www.ctpublic.org/news/2023-07-14/connecticut-river-seeing-water-quality-issues-amid-flooding


### Citation
U.S. Geological Survey, 2023, National Water Information System data available on the World Wide Web (USGS Water Data for the Nation), accessed September 16, 2023, at URL https://waterdata.usgs.gov/nwis/dv?referred_module%253Dsw%2526site_no%253D01184000

In [11]:
%%capture
%%bash
jupyter nbconvert streamflow_time_series.ipynb --to html --no-input